# Hand Gesture With Recurrent Neural Network
#### In this file we will see the TensorFlow-Keras implementation of two recurrent neural networks for the recognition of dynamic hand gestures. We will use American Sign Language (ASL) Dataset for training and testing of our model. We will train the following two bidirectional recurrent neural networkd models.
#### a) Bidirectional Gated Recurrent Unit (Bi-GRU)
#### b) Bidirectional Long Short Term Memory (Bi-LSTM)

### Building Model

#### Importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow import keras
from keras.layers import Layer
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional, Input, TimeDistributed, CuDNNLSTM, CuDNNGRU
from keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
import pickle
from sklearn.utils import shuffle
from tensorflow.keras.layers.experimental.preprocessing import Normalization
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

### Bidirectional RNN models

#### Parameeters

In [ ]:
units = 200
n_classes = 30
dropout = 0.2
attributes = 32

In [ ]:
#--------------------------------------------------------------------
# Creating GRU Model
#--------------------------------------------------------------------
def build_gru_model(units, attributes, dropout, n_classes):
    # Functional API based model
    inputs = Input(shape=(units, attributes))
    gru1 = Bidirectional(GRU(units, activation='tanh', dropout=dropout, return_sequences=True))(inputs)
    gru2 = Bidirectional(GRU(units, activation='tanh', dropout=dropout))(gru1)
    output = Dense(n_classes, activation='softmax')(gru2)
    model = Model(inputs, output)
    return model

In [ ]:
#--------------------------------------------------------------------
# Creating LSTM Model
#--------------------------------------------------------------------
def build_lstm_model(units, attributes, dropout, n_classes):
    # Functional API based model
    inputs = Input(shape=(units, attributes))
    gru1 = Bidirectional(LSTM(units, activation='tanh', dropout=dropout, return_sequences=True))(inputs)
    gru2 = Bidirectional(LSTM(units, activation='tanh', dropout=dropout))(gru1)
    output = Dense(n_classes, activation='softmax')(gru2)
    model = Model(inputs, output)
    return model

In [ ]:
model = build_gru_model(units, attributes, dropout, n_classes)
# model = build_lstm_model(units, attributes, dropout, n_classes)

In [ ]:
model.summary()

### Loading dataset

In [ ]:
def load_data(path):
    '''
    Returns train(x_train) and test(x_test) hand gesture sequences and 
    their associated labels (y_train), (y_test) respectively
    '''
    file = open(path, 'rb')
    data = pickle.load(file, encoding='latin1')  # <<---- change to 'latin1' to 'utf8' if the data does not load
    file.close()
    return data['x_train'], data['x_test'], data['y_train'], data['y_test']

In [ ]:
#----------------------------------------------------------------------
# load data from pickle file
#----------------------------------------------------------------------
pickle_file_path = 'saved_datasets_pickle/asl.pckle'
x_train, x_test, y_train, y_test = load_data(path=pickle_file_path)

In [ ]:
#----------------------------------------------------------------------
# Data Shuffle
#----------------------------------------------------------------------  
x_train, y_train= shuffle(x_train, y_train, random_state= 10)
x_test, y_test= shuffle(x_test, y_test, random_state = 0)

In [ ]:
#----------------------------------------------------------------------
# Procesing data before giving to the model
#----------------------------------------------------------------------    
def normalize_data(x_train, x_test):
    normalize = Normalization()
    normalize.adapt(x_train)
    x_train = np.asarray(normalize(x_train))
    print('var_normalized_train : %.10f' % np.var(x_train))
    print('mean_normalized_train : %.10f' % np.mean(x_train))
    normalize.adapt(x_test)
    x_test = np.asarray(normalize(x_test))
    print('var_normalized_test : %.10f' % np.var(x_test))
    print('mean_normalized_test : %.10f' % np.mean(x_test))
        
    return x_train, x_test

In [ ]:
x_train, x_test = normalize_data(x_train, x_test)

### Model training

In [ ]:
# learning rate
lr_rate = 0.0001
# optimize used for classification
opt = Adam(learning_rate=lr_rate)
loss = 'categorical_crossentropy'
metrics = ['accuracy']

model.compile(loss=loss, optimizer=opt, metrics=metrics)

In [ ]:
#----------------------------------------------------------------------
# model training
#----------------------------------------------------------------------  
# validation set is taken from train data
history = model.fit(x_train, y_train, batch_size=100, epochs=100, shuffle=True, validation_split=0.2, verbose=1)

# if there is a validation set
# history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=100)

### Saving model and history

In [ ]:
saved_model_path = 'testing_models\my_model_00'
model.save(saved_model_path)

In [ ]:
# save:
f = open('testing_models\history_saved_416.pckl', 'wb')
pickle.dump(history.history, f)
f.close()

### Loadind saved model and history

In [ ]:
loaded_model = keras.models.load_model('testing_models\my_model_423')

In [ ]:
# retrieve:    
f = open('testing_models\history_saved_423.pckl', 'rb')
history = pickle.load(f)
f.close()

### Model evaluation

In [ ]:
#----------------------------------------------------------------------
# model evaluation on test data and measure on accuracy metrics
#----------------------------------------------------------------------  
score = loaded_model.evaluate(x_test, y_test, batch_size=100, verbose=1)
print("Test Score:", score[0])
print("Accuracy: %.2f%%" % (score[1]*100))

### Ploting curves

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','validation'], loc='lower right')
plt.show()

plt.plot(history['loss'])
plt.plot(history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','validation'], loc='upper right')
plt.show()

### Confusion matrix

In [ ]:
def plot_confusion_matrix(y_test, y_scores, classNames):
    y_test=np.argmax(y_test, axis=1)
    y_scores=np.argmax(y_scores, axis=1)
    classes = len(classNames)
    cm = confusion_matrix(y_test, y_scores)
    print('** Confusion Matrix **')
    # print("Classification Report")
    # print(classification_report(y_test, y_scores, target_names=classNames))
    con = np.zeros((classes,classes))
    for x in range(classes):
        for y in range(classes):
            con[x,y] = cm[x,y]/np.sum(cm[x,:])

    plt.figure(figsize=(14, 10))
    sns.set(font_scale=0.8) # for label size
    df = sns.heatmap(con, annot=True,fmt='.1f', cmap='Blues',xticklabels= classNames , yticklabels= classNames)
#     df.figure.savefig("Results\GRU_W_with_X_train(no feature scaling)\confusion matrix\confusion_bi_124.png")

In [ ]:
classNames = ['0','1','2','3','4','5','6','7','8','9','10',
                  '11','12','13','14','15','16','17','18','19',
                  '20','21','22','23','24','25','26','27','28','29']  

predictions = loaded_model.predict(x_test)

plot_confusion_matrix(y_test, predictions, classNames)